## Importing dependencies and environmental variables

In [3]:
# Dependencies
import csv as csv
import http.client 
import json
import numpy as np
import os
import pandas as pd
import pprint
import psycopg2
import requests
import sqlalchemy
from bs4 import BeautifulSoup as bs
from dotenv import load_dotenv
from selenium import webdriver
from sodapy import Socrata
from splinter import Browser
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, Float, Date

In [4]:
# Loading homeowrk7.env so that the environment variables can be used in the engine object url
load_dotenv("project3.env")
# Initializing variables to hold each environmet varaible
username=os.environ.get("USERNAME")
password=os.environ.get("PASSWORD")
host=os.environ.get("HOST")
port=os.environ.get("PORT")
database=os.environ.get("DATABASE")
google_key = os.environ.get('GOOGLE_API_KEY')
yelp_client_id = os.environ.get('YELP_CLIENT_ID')
yelp_key = os.environ.get('YELP_API_KEY')
sf_data_key = os.environ.get('SFDATAAPPTOKEN')

In [5]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

## Querying the Seattle Registered Business API for Startbucks locations

In [4]:
client = Socrata("data.seattle.gov", sf_data_key)

In [31]:
starbucks_results = client.get("wnbq-64tb", where="business_legal_name ='STARBUCKS CORPORATION'")
starbucks_results_df = pd.DataFrame.from_records(starbucks_results)

## Querying the Seattle Registered Business API for Peet's Coffee locations

In Seattle Peet's coffee is sold at a coffee shop and bakery called Specialty's Cafe and Bakery. We will have to decide whether we want to include this in our data set.

In [53]:
specialtys_coffee_results = client.get("wnbq-64tb", where="trade_name LIKE'%SPECIALTYS CAFE%'")
specialtys_coffee_results_df = pd.DataFrame.from_records(specialtys_coffee_results)

## Querying the Seattle Registered Business API for bourgie coffee shops

In [104]:
bourgie_coffee_shop_list = ["PRESERVE AND GATHER",
                               "MAKEDA AND MINGUS",
                               "HERKIMER COFFEE",
                               "ZOKA",
                               "VIF",
                               "LIGHTHOUSE ROASTERS",
                               "MILSTEAD",
                               "CAFFE LADRO",
                               "ESPRESSO VIVACE",
                               "LA MARZOCCO EXPERIENCE",
                               "ANALOG COFFEE",
                               "MR WEST",
                               "VICTROLA",
                               "CAFFE VITA",
                               "TOUGO",
                               "SEATTLE COFFEE WORKS",
                               "ELM COFFEE",
                               "CAFFE UMBRIA",
                               "SOUND & FOG",
                               "EMPIRE ESPRESSO"
                              ]

In [107]:
bourgie_coffee_df = pd.DataFrame()
for coffee_shop in bourgie_coffee_shop_list:
    query = client.get("wnbq-64tb", where=f"trade_name LIKE '%{coffee_shop}%'")
    results_df = pd.DataFrame.from_records(query)
    bourgie_coffee_df = pd.concat([bourgie_coffee_df, results_df], ignore_index=True, sort=True)

## Concatenating all coffee shop data frames together and cleaning them

In [108]:
complete_coffee_df = pd.DataFrame()
complete_coffee_df = pd.concat([starbucks_results_df, 
                                bourgie_coffee_df
                               ], ignore_index=True, sort=True)

In [109]:
complete_coffee_df

,business_legal_name,business_phone,city_account_number,city_state_zip,license_start_date,naics_code,naics_description,ownership_type,state,street_address,trade_name,ubi,zip
0,STARBUCKS CORPORATION,206-555-1212,0001054440643765,SEATTLE,2010-07-01T00:00:00.000,541990,"All Other Professional, Scientific, and Techni...",Corporation,WA,700 5TH,BOARD OF DIRECTORS,6006111090010001,98104
1,STARBUCKS CORPORATION,206-318-8705,0001054440585950,SEATTLE,2005-05-06T00:00:00.000,722513,Limited-Service Restaurants,Corporation,WA,1962 1ST AVE S,STARBUCKS COFFEE 9303,6006111090010369,98134
2,STARBUCKS CORPORATION,206-447-1575,0001054440106297,SEATTLE,1993-08-14T00:00:00.000,722513,Limited-Service Restaurants,Corporation,WA,2401 UTAH AVE S,STARBUCKS COFFEE 100,6006111090010120,98134
3,STARBUCKS CORPORATION,206-555-1212,0001054440598744,SEATTLE,2006-09-12T00:00:00.000,722513,Limited-Service Restaurants,Corporation,WA,701 5TH AVE # 4020,STARBUCKS COFFEE 10302,6006111090010422,98104
4,STARBUCKS CORPORATION,206-447-1575,0001054440106279,SEATTLE,1990-01-01T00:00:00.000,722513,Limited-Service Restaurants,Corporation,WA,2101 4TH AVE # 150,STARBUCKS COFFEE 106,6006111090010023,98121
5,STARBUCKS CORPORATION,206-447-1575,0001054440106280,SEATTLE,1990-01-01T00:00:00.000,722513,Limited-Service Restaurants,Corporation,WA,1420 5TH AVE,STARBUCKS COFFEE 109 KIOSK,6006111090010011,98101
6,STARBUCKS CORPORATION,206-318-4994,0001054440595075,SEATTLE,2006-06-23T00:00:00.000,722513,Limited-Service Restaurants,Corporation,WA,1301 2ND AVE #2ND FL,STARBUCKS COFFEE 10927,6006111090010403,98101
7,STARBUCKS CORPORATION,206-447-1575,0001054440106292,SEATTLE,1994-03-26T00:00:00.000,722513,Limited-Service Restaurants,Corporation,WA,1101 E MADISON ST,STARBUCKS COFFEE 112,6006111090010098,98104
8,STARBUCKS CORPORATION,206-625-1255,0001054440106293,SEATTLE,1994-03-28T00:00:00.000,722513,Limited-Service Restaurants,Corporation,WA,1501 4TH AVE,STARBUCKS COFFEE 113,6006111090010097,98101
9,STARBUCKS CORPORATION,206-447-1575,0001054440106296,SEATTLE,1994-11-01T00:00:00.000,722513,Limited-Service Restaurants,Corporation,WA,600 UNIVERSITY ST,STARBUCKS COFFEE 114,6006111090010114,98101


In [110]:
complete_coffee_df.to_csv("data/seattle_coffee_shop_data.csv", index=False, header=True)

## Formatting all result .csv's to have the same headers so they can be concatanated

In [39]:
complete_coffee_df = pd.read_csv("data/seattle_coffee_shop_data.csv")

In [40]:
complete_coffee_df.columns

Index(['business_legal_name', 'business_phone', 'city_account_number', 'city_state_zip', 'license_start_date', 'naics_code', 'naics_description', 'ownership_type', 'state', 'street_address', 'trade_name', 'ubi', 'zip'], dtype='object')

In [41]:
complete_coffee_df["location"] = None
complete_coffee_df["location_end_date"] = None

In [42]:
cleaned_complete_coffee_df = complete_coffee_df[["zip",
                                                 "trade_name",
                                                 "business_legal_name",
                                                 "city_state_zip",
                                                 "street_address",
                                                 "location",
                                                 "license_start_date",   
                                                 "location_end_date"
                                                ]]

In [43]:
cleaned_complete_coffee_df

,zip,trade_name,business_legal_name,city_state_zip,street_address,location,license_start_date,location_end_date
0,98104,BOARD OF DIRECTORS,STARBUCKS CORPORATION,SEATTLE,700 5TH,None,2010-07-01T00:00:00.000,None
1,98134,STARBUCKS COFFEE 9303,STARBUCKS CORPORATION,SEATTLE,1962 1ST AVE S,None,2005-05-06T00:00:00.000,None
2,98134,STARBUCKS COFFEE 100,STARBUCKS CORPORATION,SEATTLE,2401 UTAH AVE S,None,1993-08-14T00:00:00.000,None
3,98104,STARBUCKS COFFEE 10302,STARBUCKS CORPORATION,SEATTLE,701 5TH AVE # 4020,None,2006-09-12T00:00:00.000,None
4,98121,STARBUCKS COFFEE 106,STARBUCKS CORPORATION,SEATTLE,2101 4TH AVE # 150,None,1990-01-01T00:00:00.000,None
5,98101,STARBUCKS COFFEE 109 KIOSK,STARBUCKS CORPORATION,SEATTLE,1420 5TH AVE,None,1990-01-01T00:00:00.000,None
6,98101,STARBUCKS COFFEE 10927,STARBUCKS CORPORATION,SEATTLE,1301 2ND AVE #2ND FL,None,2006-06-23T00:00:00.000,None
7,98104,STARBUCKS COFFEE 112,STARBUCKS CORPORATION,SEATTLE,1101 E MADISON ST,None,1994-03-26T00:00:00.000,None
8,98101,STARBUCKS COFFEE 113,STARBUCKS CORPORATION,SEATTLE,1501 4TH AVE,None,1994-03-28T00:00:00.000,None
9,98101,STARBUCKS COFFEE 114,STARBUCKS CORPORATION,SEATTLE,600 UNIVERSITY ST,None,1994-11-01T00:00:00.000,None


In [44]:
cleaned_complete_coffee_df = cleaned_complete_coffee_df.rename(columns={"zip": "zip_code",
                                                                        "trade_name": "dba_name",
                                                                        "business_legal_name": "ownership_name",
                                                                        "city_state_zip": "city",
                                                                        "license_start_date": "location_start_date"
                                                                        })

In [45]:
cleaned_complete_coffee_df

,zip_code,dba_name,ownership_name,city,street_address,location,location_start_date,location_end_date
0,98104,BOARD OF DIRECTORS,STARBUCKS CORPORATION,SEATTLE,700 5TH,None,2010-07-01T00:00:00.000,None
1,98134,STARBUCKS COFFEE 9303,STARBUCKS CORPORATION,SEATTLE,1962 1ST AVE S,None,2005-05-06T00:00:00.000,None
2,98134,STARBUCKS COFFEE 100,STARBUCKS CORPORATION,SEATTLE,2401 UTAH AVE S,None,1993-08-14T00:00:00.000,None
3,98104,STARBUCKS COFFEE 10302,STARBUCKS CORPORATION,SEATTLE,701 5TH AVE # 4020,None,2006-09-12T00:00:00.000,None
4,98121,STARBUCKS COFFEE 106,STARBUCKS CORPORATION,SEATTLE,2101 4TH AVE # 150,None,1990-01-01T00:00:00.000,None
5,98101,STARBUCKS COFFEE 109 KIOSK,STARBUCKS CORPORATION,SEATTLE,1420 5TH AVE,None,1990-01-01T00:00:00.000,None
6,98101,STARBUCKS COFFEE 10927,STARBUCKS CORPORATION,SEATTLE,1301 2ND AVE #2ND FL,None,2006-06-23T00:00:00.000,None
7,98104,STARBUCKS COFFEE 112,STARBUCKS CORPORATION,SEATTLE,1101 E MADISON ST,None,1994-03-26T00:00:00.000,None
8,98101,STARBUCKS COFFEE 113,STARBUCKS CORPORATION,SEATTLE,1501 4TH AVE,None,1994-03-28T00:00:00.000,None
9,98101,STARBUCKS COFFEE 114,STARBUCKS CORPORATION,SEATTLE,600 UNIVERSITY ST,None,1994-11-01T00:00:00.000,None


In [49]:
cleaned_complete_coffee_df["dba_name"] = cleaned_complete_coffee_df["dba_name"].str.title()
cleaned_complete_coffee_df["ownership_name"] = cleaned_complete_coffee_df["ownership_name"].str.title()
cleaned_complete_coffee_df["city"] = cleaned_complete_coffee_df["city"].str.title()
cleaned_complete_coffee_df["street_address"] = cleaned_complete_coffee_df["street_address"].str.title()

In [ ]:
for row in cleaned_complete_coffee_df.iterrows():
    row[1][0] = int(row[1][0][:5])

In [59]:
cleaned_complete_coffee_df

,zip_code,dba_name,ownership_name,city,street_address,location,location_start_date,location_end_date
0,98104,Board Of Directors,Starbucks Corporation,Seattle,700 5Th,None,2010-07-01T00:00:00.000,None
1,98134,Starbucks Coffee 9303,Starbucks Corporation,Seattle,1962 1St Ave S,None,2005-05-06T00:00:00.000,None
2,98134,Starbucks Coffee 100,Starbucks Corporation,Seattle,2401 Utah Ave S,None,1993-08-14T00:00:00.000,None
3,98104,Starbucks Coffee 10302,Starbucks Corporation,Seattle,701 5Th Ave # 4020,None,2006-09-12T00:00:00.000,None
4,98121,Starbucks Coffee 106,Starbucks Corporation,Seattle,2101 4Th Ave # 150,None,1990-01-01T00:00:00.000,None
5,98101,Starbucks Coffee 109 Kiosk,Starbucks Corporation,Seattle,1420 5Th Ave,None,1990-01-01T00:00:00.000,None
6,98101,Starbucks Coffee 10927,Starbucks Corporation,Seattle,1301 2Nd Ave #2Nd Fl,None,2006-06-23T00:00:00.000,None
7,98104,Starbucks Coffee 112,Starbucks Corporation,Seattle,1101 E Madison St,None,1994-03-26T00:00:00.000,None
8,98101,Starbucks Coffee 113,Starbucks Corporation,Seattle,1501 4Th Ave,None,1994-03-28T00:00:00.000,None
9,98101,Starbucks Coffee 114,Starbucks Corporation,Seattle,600 University St,None,1994-11-01T00:00:00.000,None


In [60]:
cleaned_complete_coffee_df.to_csv("data/cleaned_seattle_coffee_shop_data.csv", index=False, header=True)